# Обучение Wake Word модели в Google Colab

Этот ноутбук поможет вам обучить собственную модель пробуждения (wake word) используя openwakeword.

## 1. Проверка GPU

Убедитесь, что GPU включён: **Runtime → Change runtime type → GPU**

In [ ]:
# Проверяем доступность GPU
!nvidia-smi

## 2. Клонирование репозитория

In [ ]:
# Клонируем репозиторий
!git clone https://github.com/dtarasyuk369/train-wake-word-model.git
%cd train-wake-word-model

## 3. Настройка окружения

Используем скрипт `setup.sh` для автоматической установки всех зависимостей:
- Python пакеты
- piper-phonemize
- piper-sample-generator
- TTS модель
- OpenWakeWord модели

In [ ]:
# Запускаем setup.sh с флагом --no-venv для Colab
# (в Colab не нужно виртуальное окружение)
!bash scripts/setup.sh --no-venv --python python3

## 4. Скачивание данных

Используем скрипт `download_data.sh` для скачивания:
- Предобработанных признаков ACAV100M
- Validation set features
- MIT RIRs (импульсные характеристики помещений)
- AudioSet 16kHz (фоновый шум)

In [ ]:
# Скачиваем данные с уменьшенным количеством AudioSet клипов для Colab
!bash scripts/download_data.sh --no-venv --data-dir ./data

# Опционально: скачать FMA для музыкального фона (~7 ГБ, займёт дополнительное время)
# !python download_data.py --include-fma --fma-hours 1

## 5. Настройка конфигурации

Создадим файл конфигурации для обучения. Измените параметры под свои нужды:

In [ ]:
import yaml
import os

# ============================================
# НАСТРОЙТЕ ЭТИ ПАРАМЕТРЫ ПОД СВОЮ МОДЕЛЬ
# ============================================

config = {
    'model_name': 'my_wake_word',  # 🔧 Измените на своё имя модели
    'target_phrase': ['hey assistant'],  # 🔧 Ваше пробуждающее слово/фраза
    
    # Опционально: добавьте кастомные негативные фразы
    'custom_negative_phrases': [],
    
    # Количество примеров (уменьшено для Colab)
    'n_samples': 3000,
    'n_samples_val': 1000,
    
    # Размеры батчей
    'tts_batch_size': 32,
    'augmentation_batch_size': 16,
    
    # Пути (не изменяйте, если не знаете что делаете)
    'piper_sample_generator_path': './piper-sample-generator',
    'piper_model_path': './piper-sample-generator/models/en_US-libritts_r-medium.pt',
    'output_dir': './my_custom_model',
    'rir_paths': ['./data/mit_rirs'],
    'background_paths': ['./data/audioset_16k'],
    'background_paths_duplication_rate': [1],
    'false_positive_validation_data_path': './data/validation_set_features.npy',
    'augmentation_rounds': 1,
    
    'feature_data_files': {
        'ACAV100M_sample': './data/openwakeword_features_ACAV100M_2000_hrs_16bit.npy'
    },
    'batch_n_per_class': {
        'ACAV100M_sample': 1024,
        'adversarial_negative': 50,
        'positive': 50
    },
    
    # Параметры модели
    'model_type': 'dnn',
    'layer_size': 32,  # Увеличьте до 64 или 128 для лучшего качества
    
    # Параметры обучения (уменьшено для Colab)
    'steps': 5000,  # Увеличьте до 10000+ для лучшего качества
    'max_negative_weight': 1500,
    'target_false_positives_per_hour': 0.2,
    'target_accuracy': 0.8,
    'target_recall': 0.6
}

# Создаём директорию для конфига если не существует
os.makedirs('./config', exist_ok=True)

# Сохраняем конфигурацию
config_path = './config/colab_config.yaml'
with open(config_path, 'w') as f:
    yaml.dump(config, f)

print(f"✅ Конфигурация создана: {config_path}")
print(f"\n📋 Параметры модели:")
print(f"  • Имя модели: {config['model_name']}")
print(f"  • Фраза: {config['target_phrase']}")
print(f"  • Примеры: {config['n_samples']} обучающих, {config['n_samples_val']} валидационных")
print(f"  • Шагов обучения: {config['steps']}")
print(f"  • Размер слоя: {config['layer_size']}")

## 6. Обучение модели

### Вариант 1: Автоматический полный цикл (рекомендуется)

Используем скрипт `train.sh` который автоматически выполнит все этапы:
1. Генерация синтетических образцов
2. Аугментация данных
3. Обучение модели

In [ ]:
# Полный автоматический цикл обучения
!bash scripts/train.sh --no-venv --config ./config/colab_config.yaml

### Вариант 2: Пошаговое выполнение (для контроля)

Если вы хотите контролировать каждый этап отдельно:

In [ ]:
# Шаг 1: Генерация синтетических образцов
!python train.py --training_config ./config/colab_config.yaml --generate_clips

In [ ]:
# Шаг 2: Аугментация данных
!python train.py --training_config ./config/colab_config.yaml --augment_clips

In [ ]:
# Шаг 3: Обучение модели
!python train.py --training_config ./config/colab_config.yaml --train_model

## 7. Скачивание обученной модели

Скачайте модель на локальный компьютер:

In [ ]:
from google.colab import files
import os

# Путь к вашей модели
model_name = config['model_name']
model_path = f"./my_custom_model/{model_name}.onnx"

if os.path.exists(model_path):
    print(f"✅ Модель найдена: {model_path}")
    print(f"📊 Размер модели: {os.path.getsize(model_path) / 1024:.2f} KB")
    files.download(model_path)
    print(f"\n✅ Модель {model_name}.onnx скачана!")
else:
    print(f"❌ Модель не найдена по пути: {model_path}")
    print("\n🔍 Проверяем содержимое директории:")
    !ls -la ./my_custom_model/

## 8. Тестирование модели (опционально)

Протестируйте модель для проверки работоспособности:

In [ ]:
import openwakeword
from openwakeword.model import Model
import numpy as np

# Загружаем модель
if os.path.exists(model_path):
    owwModel = Model(wakeword_models=[model_path])
    print(f"✅ Модель {model_name} загружена!")
    print(f"\n📊 Информация о модели:")
    print(f"  • Размер: {os.path.getsize(model_path) / 1024:.2f} KB")
    print(f"  • Путь: {model_path}")
    print(f"\n✅ Модель готова к использованию!")
else:
    print(f"❌ Модель не найдена: {model_path}")

## 9. Сохранение в Google Drive (опционально)

Сохраните модель и конфигурацию в Google Drive для повторного использования:

In [ ]:
from google.colab import drive
import shutil

# Монтируем Google Drive
drive.mount('/content/drive')

# Создаём директорию в Drive
drive_dir = '/content/drive/MyDrive/wake_word_models'
os.makedirs(drive_dir, exist_ok=True)

# Копируем модель
if os.path.exists(model_path):
    shutil.copy(model_path, drive_dir)
    print(f"✅ Модель скопирована в Google Drive: {drive_dir}")
    
    # Копируем конфигурацию
    shutil.copy(config_path, os.path.join(drive_dir, f"{model_name}_config.yaml"))
    print(f"✅ Конфигурация также сохранена")
    
    print(f"\n📁 Файлы в Google Drive:")
    !ls -lh "{drive_dir}"

## 10. Очистка (опционально)

Освободите место, удалив промежуточные файлы:

In [ ]:
# Раскомментируйте нужные команды для очистки:

# Удалить фоновые данные (освободить ~5-10 ГБ)
# !rm -rf ./data/audioset_16k
# !rm -rf ./data/fma

# Удалить промежуточные аудио файлы (освободить ~2-5 ГБ)
# !rm -rf ./my_custom_model/*/positive_train
# !rm -rf ./my_custom_model/*/negative_train
# !rm -rf ./my_custom_model/*/positive_test
# !rm -rf ./my_custom_model/*/negative_test

# Удалить features файлы (освободить ~1-3 ГБ)
# !rm -rf ./my_custom_model/*/*.npy

print("ℹ️ Для удаления файлов раскомментируйте нужные команды выше")

---

## 🎉 Готово!

Ваша модель обучена и готова к использованию. Следующие шаги:

1. **Скачайте** `.onnx` файл на локальный компьютер
2. **Интегрируйте** в своё приложение используя OpenWakeWord
3. **Протестируйте** на реальных условиях
4. **Дообучите** если нужно улучшить качество

### 📝 Использование модели:

```python
from openwakeword.model import Model
import pyaudio
import numpy as np

# Загрузить модель
model = Model(wakeword_models=["your_model.onnx"])

# Использовать с микрофоном
audio = pyaudio.PyAudio()
stream = audio.open(format=pyaudio.paInt16, channels=1, 
                    rate=16000, input=True, frames_per_buffer=1280)

while True:
    data = np.frombuffer(stream.read(1280), dtype=np.int16)
    prediction = model.predict(data)
    if max(prediction.values()) > 0.5:
        print("Wake word detected!")
```

### 🔗 Полезные ссылки:

- 📖 [OpenWakeWord документация](https://github.com/dscripka/openWakeWord)
- 📖 [Инструкции для Colab](https://github.com/dtarasyuk369/train-wake-word-model/blob/main/COLAB_INSTRUCTIONS.md)
- 📖 [README проекта](https://github.com/dtarasyuk369/train-wake-word-model/blob/main/README.md)
- 🐛 [Сообщить о проблеме](https://github.com/dtarasyuk369/train-wake-word-model/issues)

### 💡 Советы по улучшению модели:

- Увеличьте `n_samples` и `steps` для лучшего качества
- Добавьте больше разнообразного фонового шума
- Используйте `layer_size: 64` или `128` для более сложной модели
- Настройте `max_negative_weight` для баланса точности
- Добавьте кастомные негативные фразы в `custom_negative_phrases`